In [ ]:
import pandas as pd
import numpy as np
from math import floor
import math
import os
from datetime import datetime
import xarray as xr

In [2]:
# DF con playas
# df_playas = pd.read_excel('./AvistamientosConRedondeoCoord-025.xlsx').reset_index()
df_playas = pd.read_excel('./AvistamientosConRedondeoCoord-00833.xlsx').reset_index()
'''
Las celdas que estaban unidas por tener el mismo valor, tienen valores nulos
por lo que las relleno con us valor correspodiente
'''
df_playas = df_playas.fillna(method= 'ffill',axis=0)
# Cojo las columnas que me interesan
df_playas = df_playas[["Suma","Lat_floor","Long_floor","Fecha"]]
# Renombro las columnas 
df_playas = df_playas.rename(columns={"Suma": "Avistamientos", "Lat_floor": "Latitud", "Long_floor": "Longitud"})
# Ordeno por fecha
df_playas = df_playas.sort_values(by=["Fecha"])

In [3]:
df_playas # DataFrame con avistamientos en cada playa
cuadrantes = { # Numero de cuadrantes que se concatenarán con las playas
    'izquierda': 0 ,
    'arriba': 0,
    'abajo': 0
}
salto = 1/12 # Salto de coordenadas entre los cuadrantes
listado_archivos = os.listdir('../../descargas') # Listo todos los archivos de Copernicus
variables = ['Coord','Profundidad','mlotst','zos','bottomT','thetao','so','uo','vo']

In [4]:
df_playas # DataFrame con avistamientos en cada playa
cuadrantes = { # Numero de cuadrantes que se concatenarán con las playas
    'izquierda': 0 ,
    'arriba': 0,
    'abajo': 0
}
dias_desfase = 2
salto = 1/12 # Salto de coordenadas entre los cuadrantes
listado_archivos = os.listdir('../../descargas') # Listo todos los archivos de Copernicus
variables = ['Coord','Profundidad','mlotst','zos','bottomT','thetao','so','uo','vo']

In [5]:
def busca_archivo(fecha):
    texto ='_{}_'.format(str(fecha).split()[0].replace('-',''))
    archivo = [x for x in listado_archivos if str(texto) in x]
    data = xr.open_dataset('../../descargas/{}'.format(archivo[0])) # cargo el archivo
    return data # devuelvo dataset

def dame_datos(latitud,longitud,ds):
    return ds.sel({'latitude':latitud,'longitude': longitud})

def comprueba_datos(latitud,longitud,ds):
    valor = dame_datos(latitud,longitud,ds)
    while math.isnan(valor.mlotst[0]):
        longitud = longitud - salto
        valor = dame_datos(latitud,longitud,ds)
    return latitud,longitud # devuelvo las coordenadas con datos

def crea_DF():
    df_resultados = None
    df_resultados = pd.DataFrame(columns=["Avistamientos","Latitud","Longitud","Fecha",'Coord',"Profundidad","mlotst",'zos','bottomT','thetao','so','uo','vo']) # columnas básicas
    for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda meto un grupo de columnas
        var_aux = [str(x+'_izq_'+str(izquierda+1)) for x in variables]
        df_aux = pd.DataFrame(columns=var_aux)
        df_resultados = df_resultados.join(df_aux)
        for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior meto un grupo de columnas
            var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
        for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior meto un grupo de columnas    
            var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
    return df_resultados

def meto_en_df(df_resultados,row,coordenadas,df,indice,variables_actuales):
    for x in range(3):
        df_resultados.loc[indice + x,["Avistamientos","Latitud","Longitud","Fecha"]] = row.values.tolist() # meto en df_resulados los datos de ese cuadrante
        vars = df.loc[x,['depth', 'mlotst','zos','bottomT','thetao','so','uo','vo']].values.tolist()
        vars.insert(0,coordenadas)
        df_resultados.loc[indice + x,variables_actuales] = vars # meto coordenadas reales utilizadas y datos 
    return df_resultados

def dameCoordIzq(lat,long,n_saltos):
    return [lat,long-(salto*n_saltos)]

def dameCoordSuperior(lat,long,n_saltos):
    coordenadas = dameCoordIzq(lat,long,n_saltos)
    return [lat+(salto*n_saltos),long]

def dameCoordInferior(lat,long,n_saltos):
    return [lat-(salto*n_saltos),long]

def resta_fecha(fecha,dias):
    return fecha - pd.Timedelta(days=2)

In [6]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
        print(contador,end='\r')
        ds = busca_archivo(resta_fecha(row['Fecha'],dias_desfase)) # busco el archivo de la fecha 
        coordenadas = comprueba_datos(row['Latitud'],row['Longitud'],ds) # miro a ver si el cuadrante tiene datos, sino cojo el siguiente por la izquierda

        df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,list(coordenadas),df,contador * 3,variables)

        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)

#         if contador == 5:
#             break
    return df_resultados

In [7]:
df_resultados = crea_estr()
df_resultados.head()

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,so,uo,vo
0,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.58333333333333]",0.494025,10.5289,0.166326,12.3865,15.3455,34.7255,-0.104373,0.0286874
1,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.58333333333333]",5.07822,10.5289,0.166326,12.3865,15.2906,34.7255,-0.111087,-0.00183111
2,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666666, -70.58333333333333]",9.573,10.5289,0.166326,12.3865,15.1727,34.7209,-0.112308,-0.028077
3,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",0.494025,10.5289,0.21485,15.2723,17.7106,34.8735,-0.0579852,0.126957
4,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",5.07822,10.5289,0.21485,15.2723,17.3605,34.8949,-0.0518815,0.0787378


In [8]:
df_resultados = df_resultados.reset_index().set_index(['Latitud','Longitud','Fecha','Avistamientos','Profundidad'])
df_resultados.to_excel('EstructuraFinalDesfaseFecha.xlsx')
df_resultados.head()

index  \
Latitud    Longitud   Fecha      Avistamientos Profundidad          
-25.416667 -70.500000 2014-05-01 1             0.494025         0   
                                               5.078224         1   
                                               9.572997         2   
-18.500000 -70.333333 2014-05-01 1             0.494025         3   
                                               5.078224         4   

                                                                                               Coord  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                                             
-25.416667 -70.500000 2014-05-01 1             0.494025     [-25.41666666666666, -70.58333333333333]   
                                               5.078224     [-25.41666666666666, -70.58333333333333]   
                                               9.572997     [-25.41666666666666, -70.58333333333333]   
-18.500000 -70.333333 2014-05-01 1             0.494025                  [-18.5, -70.33333333333333]   
                                               5.078224                  [-18.5, -70.33333333333333]   

                                                             mlotst       zos  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                      
-25.416667 -70.500000 2014-05-01 1             0.494025     10.5289  0.166326   
                                               5.078224     10.5289  0.166326   
                                               9.572997     10.5289  0.166326   
-18.500000 -70.333333 2014-05-01 1             0.494025     10.5289   0.21485   
                                               5.078224     10.5289   0.21485   

                                                            bottomT   thetao  \
Latitud    Longitud   Fecha      Avistamientos Profundidad                     
-25.416667 -70.500000 2014-05-01 1             0.494025     12.3865  15.3455   
                                               5.078224     12.3865  15.2906   
                                               9.572997     12.3865  15.1727   
-18.500000 -70.333333 2014-05-01 1             0.494025     15.2723  17.7106   
                                               5.078224     15.2723  17.3605   

                                                                 so  \
Latitud    Longitud   Fecha      Avistamientos Profundidad            
-25.416667 -70.500000 2014-05-01 1             0.494025     34.7255   
                                               5.078224     34.7255   
                                               9.572997     34.7209   
-18.500000 -70.333333 2014-05-01 1             0.494025     34.8735   
                                               5.078224     34.8949   

                                                                   uo  \
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-25.416667 -70.500000 2014-05-01 1             0.494025     -0.104373   
                                               5.078224     -0.111087   
                                               9.572997     -0.112308   
-18.500000 -70.333333 2014-05-01 1             0.494025    -0.0579852   
                                               5.078224    -0.0518815   

                                                                    vo  
Latitud    Longitud   Fecha      Avistamientos Profundidad              
-25.416667 -70.500000 2014-05-01 1             0.494025      0.0286874  
                                               5.078224    -0.00183111  
                                               9.572997      -0.028077  
-18.500000 -70.333333 2014-05-01 1             0.494025       0.126957  
                                               5.078224      0.0787378

In [9]:
pd.Timedelta(days=2)

Timedelta('2 days 00:00:00')